# Funciones ventana en PySpark

Las funciones ventana se utilizan para calcular resultados como el rango, el número de fila, etc., sobre un rango de filas de entrada y devuelven un valor para cada una de ellas. Son útiles cuando necesitamos hacer operaciones de agregación en un marco de ventana específico en las columnas de un DataFrame.

PySpark soporta 3 tipos de funciones ventana:

* De ranking
* Analíticas
* De agregación

Para desarrollar una operación en un grupo primero necesitamos particionar los datos usando el método `Window.partitionBy()` y para las funciones de ranking `row_number()` y `rank()` también necesitamos ordenar por los datos particionados usando la cláusula `orderBy`.

A continuación vamos a cómo funcionan con un DataFrame de ejemplo:

In [1]:
# Import libraries needed
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [2]:
# Build a SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# Print SparkSession
spark

In [3]:
simpleData = (
    ("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
)

columns = ["employee_name", "department", "salary"]

# Create DataFrame
df = spark.createDataFrame(data = simpleData, schema = columns)

df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



## Funciones ventana de ranking

### row_number()

La función de ventana `row_number()` se utiliza para enumerar de forma secuencial empezando por 1 el grupo de filas pertenecientes a cada partición de la ventana. 

En este ejemplo particionamos por la columna `department` e indicamos que enumere por orden ascendente de `salario` cada departamento.

In [4]:
windowSpec = Window.partitionBy("department").orderBy("salary")

df.withColumn("row_number", row_number().over(windowSpec)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
+-------------+----------+------+----------+



### rank()

La función de ventana `rank()` se utiliza para proporcionar una clasificación dentro de una ventana de partición. En caso de empate entre valores de una misma partición se les asigna el mismo rango y se omite los rangos siguientes. Es decir, si tenemos 3 elementos en el rango 2, el siguiente rango sería 5.

En nuestro ejemplo vemos que en el grupo de filas del departamento <b>Ventas</b>, el empleado Robert y Saif tienen el mismo salario luego se clasifican ambos con el rango 3 en lugar de 3 y 4. 

El rango 4 se salta y el siguiente empleado Michael se clasifica con el rango 5.


In [5]:
df.withColumn("rank", rank().over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
+-------------+----------+------+----+



### dense_rank()

La función de ventana `dense_rank()` se comporta igual que `rank()` salvo que los rangos son consecutivos. Es decir, en caso de empate no se salta ningún rango.

In [6]:
df.withColumn("dense_rank", dense_rank().over(windowSpec)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
+-------------+----------+------+----------+



## Funciones ventana de agregación

Las funciones de agregación operan sobre un grupo de filas y calculan un único valor de retorno para cada grupo. Estas funciones se pueden utilizar también como función de ventana.

En este ejemplo vamos a ver como calcular el salario máximo, mínimo y total para cada departamento utilizando funciones ventana en lugar de utilizar la función `groupBy()`.


In [11]:
# Using window aggregate functions
windowSpecAgg = Window.partitionBy("department")

(
  df
  .withColumn("row", row_number().over(windowSpec))
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg))
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg))
  .withColumn("min", min(col("salary")).over(windowSpecAgg))
  .withColumn("max", max(col("salary")).over(windowSpecAgg))
  .where(col("row")==1)
  .select("department","avg","sum","min","max")
).show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|     Sales|3760.0|18800|3000|4600|
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
+----------+------+-----+----+----+



In [9]:
# Using groupBy function
(
    df
    .groupBy("department")
    .agg(
        avg("salary").alias("avg"),
        sum("salary").alias("sum"),
        min("salary").alias("min"),
        max("salary").alias("max")
    )
).show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|     Sales|3760.0|18800|3000|4600|
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
+----------+------+-----+----+----+



## Caso práctico de uso

A continuación vamos a ver un caso práctico de uso de las funciones ventana en una base de datos con más de 5 millones de registros.

La base de datos se tiene que descargar de esta [dirección]() y cargar en MySQL. 

Posteriormente, leeremos las tablas con Spark y realizaremos una serie de transformaciones para obtener un dataset limpio que guardaremos en formato parquet y utilizaremos para la práctica de las funciones ventanas.

In [10]:
#https://dev.mysql.com/doc/employee/en/
    
tables = ["employees","salaries", "titles", "dept_emp", "departments", "dept_manager"]

for table in tables:
    (spark.read
     .format("jdbc")
     .option("url", "jdbc:mysql://localhost:3306/employees")
     .option("driver", "com.mysql.jdbc.Driver")
     .option("dbtable", table)
     .option("user", "root")
     .option("password", "password")
     .load()) \
    .createOrReplaceTempView(table)
    
spark.sql("SHOW TABLES").show(truncate=False)

+--------+------------+-----------+
|database|tableName   |isTemporary|
+--------+------------+-----------+
|        |departments |true       |
|        |dept_emp    |true       |
|        |dept_manager|true       |
|        |employees   |true       |
|        |salaries    |true       |
|        |titles      |true       |
+--------+------------+-----------+



In [37]:
# Joining tables with SQL
query = """
    SELECT 
        e.emp_no,
        e.first_name || " " || e.last_name AS name,
        t.title,
        d.dept_name AS department,
        s.salary,
        s.from_date,
        s.to_date
    FROM employees AS e
    INNER JOIN titles AS t ON t.emp_no = e.emp_no
    INNER JOIN salaries AS s ON s.emp_no = e.emp_no
    INNER JOIN dept_emp AS de ON de.emp_no = e.emp_no
    INNER JOIN departments AS d ON d.dept_no = de.dept_no
"""

# Create DataFrame from query
employeeDF = spark.sql(query)

In [38]:
employeeDF.show()

+------+-----------------+----------------+-----------+------+----------+----------+
|emp_no|             name|           title| department|salary| from_date|   to_date|
+------+-----------------+----------------+-----------+------+----------+----------+
| 10206| Alassane Iwayama|Technique Leader|Development| 40000|1988-04-19|1989-04-19|
| 10206| Alassane Iwayama|Technique Leader|Development| 43519|1989-04-19|1990-04-19|
| 10206| Alassane Iwayama|Technique Leader|Development| 46265|1990-04-19|1991-04-19|
| 10206| Alassane Iwayama|Technique Leader|Development| 46865|1991-04-19|1992-04-18|
| 10206| Alassane Iwayama|Technique Leader|Development| 47837|1992-04-18|1993-04-18|
| 10206| Alassane Iwayama|Technique Leader|Development| 52042|1993-04-18|1994-04-18|
| 10206| Alassane Iwayama|Technique Leader|Development| 52370|1994-04-18|1995-04-18|
| 10206| Alassane Iwayama|Technique Leader|Development| 53202|1995-04-18|1996-04-17|
| 10206| Alassane Iwayama|Technique Leader|Development| 56087|199

In [39]:
# Write DataFrame to Parquet file
employeeDF.write.mode("overwrite").parquet("data/employee.parquet")

Una vez tenemos nuestro dataset en formato parquet vamos a leerlo con Spark y exponer el ejemplo donde podemos hacer uso de las funciones ventana:

In [10]:
employeeDF_raw = spark.read.parquet("data/employee.parquet")

employeeDF_raw.show()

+------+----------------+----------------+-----------+------+----------+----------+
|emp_no|            name|           title| department|salary| from_date|   to_date|
+------+----------------+----------------+-----------+------+----------+----------+
| 10206|Alassane Iwayama|Technique Leader|Development| 40000|1988-04-19|1989-04-19|
|450881|Irena Lieberherr|        Engineer|Development| 59264|1997-01-22|1998-01-22|
| 10206|Alassane Iwayama|Technique Leader|Development| 43519|1989-04-19|1990-04-19|
|450881|Irena Lieberherr|        Engineer|Development| 60536|1998-01-22|1999-01-22|
| 10206|Alassane Iwayama|Technique Leader|Development| 46265|1990-04-19|1991-04-19|
|450881|Irena Lieberherr|        Engineer|Development| 60170|1999-01-22|2000-01-22|
| 10206|Alassane Iwayama|Technique Leader|Development| 46865|1991-04-19|1992-04-18|
|450881|Irena Lieberherr|        Engineer|Development| 61146|2000-01-22|2001-01-21|
| 10206|Alassane Iwayama|Technique Leader|Development| 47837|1992-04-18|1993

In [11]:
# Number of records
employeeDF_raw.count()

5124191

La base de datos contiene un registro de los empleados de una empresa donde se almacena su puesto de trabajo, salario, departamento y la fecha de inicio y fin en cada puesto. Por lo tanto, cada empleado puede aparecer varias veces en la base de datos según su evolución dentro de la empresa.

Por ejemplo, si ordenamos el DataFrame por el id del empleado y la fecha del puesto más reciente vemos que el empleado nº 10001, Georgi Facello, trabaja en el departamento Development como Senior Engineer desde el 22 de Junio de 2002 hasta la fecha de hoy (se indica con el año 9999):

In [12]:
employeeDF_raw.orderBy("emp_no", desc("to_date")).show()

+------+--------------+---------------+-----------+------+----------+----------+
|emp_no|          name|          title| department|salary| from_date|   to_date|
+------+--------------+---------------+-----------+------+----------+----------+
| 10001|Georgi Facello|Senior Engineer|Development| 88958|2002-06-22|9999-01-01|
| 10001|Georgi Facello|Senior Engineer|Development| 85097|2001-06-22|2002-06-22|
| 10001|Georgi Facello|Senior Engineer|Development| 85112|2000-06-22|2001-06-22|
| 10001|Georgi Facello|Senior Engineer|Development| 84917|1999-06-23|2000-06-22|
| 10001|Georgi Facello|Senior Engineer|Development| 81097|1998-06-23|1999-06-23|
| 10001|Georgi Facello|Senior Engineer|Development| 81025|1997-06-23|1998-06-23|
| 10001|Georgi Facello|Senior Engineer|Development| 80013|1996-06-23|1997-06-23|
| 10001|Georgi Facello|Senior Engineer|Development| 76884|1995-06-24|1996-06-23|
| 10001|Georgi Facello|Senior Engineer|Development| 75994|1994-06-24|1995-06-24|
| 10001|Georgi Facello|Senio

Podemos utilizar la función de ventana `row_number()` para quedarnos con el registro de cada empleado con su puesto de trabajo más reciente.

Si particionamos por id de empleado y ordenamos por fecha descendiente observamos que la primera fila de cada partición contiene el registro más reciente para cada empleado:

In [13]:
windowSpec = Window.partitionBy("emp_no").orderBy(desc("to_date"))

employeeDF_raw.withColumn("row_number", row_number().over(windowSpec)).show()

+------+----------------+----------------+---------------+------+----------+----------+----------+
|emp_no|            name|           title|     department|salary| from_date|   to_date|row_number|
+------+----------------+----------------+---------------+------+----------+----------+----------+
| 10206|Alassane Iwayama|Technique Leader|    Development| 71052|2002-04-16|9999-01-01|         1|
| 10206|Alassane Iwayama|Technique Leader|    Development| 67588|2001-04-16|2002-04-16|         2|
| 10206|Alassane Iwayama|Technique Leader|    Development| 67944|2000-04-16|2001-04-16|         3|
| 10206|Alassane Iwayama|Technique Leader|    Development| 67137|1999-04-17|2000-04-16|         4|
| 10206|Alassane Iwayama|Technique Leader|    Development| 62716|1998-04-17|1999-04-17|         5|
| 10206|Alassane Iwayama|Technique Leader|    Development| 59252|1997-04-17|1998-04-17|         6|
| 10206|Alassane Iwayama|Technique Leader|    Development| 56087|1996-04-17|1997-04-17|         7|
| 10206|Al

De esta manera usando la función de ventana `row_number()` podemos filtrar el DataFrame por la primera fila de cada partición para así quedarnos con el registro más reciente:

In [14]:
windowSpec = Window.partitionBy("emp_no").orderBy(desc("to_date"))

employeeDF = employeeDF_raw.withColumn("row_number", row_number().over(windowSpec))\
    .where(col("row_number")==1)

employeeDF.show()

+------+------------------+----------------+------------------+------+----------+----------+----------+
|emp_no|              name|           title|        department|salary| from_date|   to_date|row_number|
+------+------------------+----------------+------------------+------+----------+----------+----------+
| 10206|  Alassane Iwayama|Technique Leader|       Development| 71052|2002-04-16|9999-01-01|         1|
| 10362|   Shalesh dAstous|    Senior Staff|   Human Resources| 54987|1996-10-31|1997-07-16|         1|
| 10623| Aleksander Danlos|        Engineer|       Development| 86399|2002-01-12|9999-01-01|         1|
| 10817|       Uri Rullman|    Senior Staff|             Sales| 78202|2001-12-23|9999-01-01|         1|
| 11033|      Shushma Bahk|        Engineer|       Development| 75271|2002-03-11|9999-01-01|         1|
| 11141| Vasiliy Kermarrec|        Engineer|       Development| 57809|2002-02-10|9999-01-01|         1|
| 11317| Shigeaki Hagimont|    Senior Staff|       Development| 

Finalmente eliminamos la columna `row_number` y obtenemos el DataFrame definitivo:

In [15]:
employeeDF = employeeDF.drop("row_number")

employeeDF.show(5)

+------+-----------------+----------------+---------------+------+----------+----------+
|emp_no|             name|           title|     department|salary| from_date|   to_date|
+------+-----------------+----------------+---------------+------+----------+----------+
| 10206| Alassane Iwayama|Technique Leader|    Development| 71052|2002-04-16|9999-01-01|
| 10362|  Shalesh dAstous|    Senior Staff|Human Resources| 54987|1996-10-31|1997-07-16|
| 10623|Aleksander Danlos|        Engineer|    Development| 86399|2002-01-12|9999-01-01|
| 10817|      Uri Rullman|    Senior Staff|          Sales| 78202|2001-12-23|9999-01-01|
| 11033|     Shushma Bahk|        Engineer|    Development| 75271|2002-03-11|9999-01-01|
+------+-----------------+----------------+---------------+------+----------+----------+
only showing top 5 rows



In [16]:
employeeDF.count()

300024

## Otros ejemplos

El siguiente dataframe muestra un extracto (ficticio) de una tabla de contratos de tarjetas de crédito que almacena datos de forma mensual, siendo el campo `closing_date` la fecha a la que hacen referencia los datos publicados.

Para cada contrato se desea obtener los datos más recientes registrados en la tabla.

In [22]:
credit_df = spark.createDataFrame([
    ('0001', '2021-04', 'Q'),
    ('0001', '2021-05', 'Q'),
    ('0001', '2021-06', None),
    ('0002', '2021-04', 'N'),
    ('0003', '2021-04', 'M'),
    ('0003', '2021-05', 'S'),
    ('0004', '2021-01', 'N'),
    ('0004', '2021-08', 'N')],
    ['contract_id', 'closing_date', 'payment_fee']
)
credit_df.show()

+-----------+------------+-----------+
|contract_id|closing_date|payment_fee|
+-----------+------------+-----------+
|       0001|     2021-04|          Q|
|       0001|     2021-05|          Q|
|       0001|     2021-06|       null|
|       0002|     2021-04|          N|
|       0003|     2021-04|          M|
|       0003|     2021-05|          S|
|       0004|     2021-01|          N|
|       0004|     2021-08|          N|
+-----------+------------+-----------+



In [27]:
w = Window.partitionBy('contract_id')
(
    credit_df.select(
        'contract_id', 
        max('closing_date').over(w).alias('closing_date'),
        'payment_fee')
    .distinct()
    .orderBy('contract_id')
).show()

+-----------+------------+-----------+
|contract_id|closing_date|payment_fee|
+-----------+------------+-----------+
|       0001|     2021-06|          Q|
|       0001|     2021-06|       null|
|       0002|     2021-04|          N|
|       0003|     2021-05|          M|
|       0003|     2021-05|          S|
|       0004|     2021-08|          N|
+-----------+------------+-----------+



Utilizando la misma tabla se quiere comprobar si un contrato ha generado comisiones en algun momento desde que está registrado en la base de datos.

Para ello se necesita analizar si el campo `payment_fee` es distinto del valor `N` (incluyendo valores nulos).

In [28]:
credit_df.show()

+-----------+------------+-----------+
|contract_id|closing_date|payment_fee|
+-----------+------------+-----------+
|       0001|     2021-04|          Q|
|       0001|     2021-05|          Q|
|       0001|     2021-06|       null|
|       0002|     2021-04|          N|
|       0003|     2021-04|          M|
|       0003|     2021-05|          S|
|       0004|     2021-01|          N|
|       0004|     2021-08|          N|
+-----------+------------+-----------+



In [45]:
tmp_df = credit_df.withColumn('fee_per_month', when(col('payment_fee') == 'N', 0).otherwise(1))

tmp_df.show()

+-----------+------------+-----------+-------------+
|contract_id|closing_date|payment_fee|fee_per_month|
+-----------+------------+-----------+-------------+
|       0001|     2021-04|          Q|            1|
|       0001|     2021-05|          Q|            1|
|       0001|     2021-06|       null|            1|
|       0002|     2021-04|          N|            0|
|       0003|     2021-04|          M|            1|
|       0003|     2021-05|          S|            1|
|       0004|     2021-01|          N|            0|
|       0004|     2021-08|          N|            0|
+-----------+------------+-----------+-------------+



In [48]:
w = Window.partitionBy('contract_id')

( 
    tmp_df.select(
        'contract_id',
        when(sum(col('fee_per_month')).over(w) > 0, 1).otherwise(0).alias('has_fee')
    )
    .distinct()
    .orderBy('contract_id')
).show()

+-----------+-------+
|contract_id|has_fee|
+-----------+-------+
|       0001|      1|
|       0002|      0|
|       0003|      1|
|       0004|      0|
+-----------+-------+



## Referencias

* https://sparkbyexamples.com/pyspark/pyspark-window-functions/